In [49]:
import os
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [50]:
#edit sesuai kebutuhan
vessel_type = 'bulk carrier'
filenum = 'bulkcarrier'
kelas = 'BKI'

In [51]:
#directory file csv
read_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/input/{}/{}.csv'.format(vessel_type,filenum)
write_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/output/{}/{}/{}.csv'.format(kelas,vessel_type,filenum)

In [52]:
# Check if input file exists
if os.path.exists(read_path):
    print(f"CSV read file is found, continuing...")
else:
    print(f"CSV read file is not found")
    print('exiting....')
    
# Check if output file exists and create it if necessary
if os.path.exists(write_path):
    print(f"CSV write file  is found, continuing...")
else:
    print(f"CSV write file is not found, creating one...")
    os.makedirs(os.path.dirname(write_path), exist_ok=True) # Create output directory if it doesn't exist
    with open(write_path, 'w', newline='') as f:
        writer = csv.writer(f)

CSV read file is found, continuing...
CSV write file  is found, continuing...


In [53]:
# create chrome options object
chrome_options = Options()

# set headless mode
chrome_options.add_argument('--headless')


driver = webdriver.Chrome(executable_path=r"chromedriver")
driver.get("https://www.bki.co.id/shipregister.html")

In [54]:
#function search di website bki
def search(name):
    vessel_name = name
    search_field = driver.find_element(By.XPATH, '/html/body/div/div/div/div/div/div/div/div[2]/form/input')
    search_field.send_keys(vessel_name)
    search_field.send_keys(Keys.RETURN)

In [55]:
#function click di website bki
def click():
    first_vessel = driver.find_element(By.XPATH, '/html/body/div/div/div/div/div/div/div/div[2]/table/tbody[2]/tr/td[1]/a')
    first_vessel.click()
    
def hull():
    hull_data = driver.find_element(By.XPATH, '/html/body/div/div/div/div/div/div/div[2]/button[2]')
    hull_data.click()

In [61]:
#roman numeral function
def roman_numeral(n):
    roman_map = {1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V', 6: 'VI', 7: 'VII', 8: 'VIII', 9: 'IX'}
    return roman_map[n] if n in roman_map else str(n)

# function iterasi nama
def process_ship_name(name, iteration):
    global vessel_name

    if iteration == 1:
    # Replace single number with Roman numeral equivalent
        name = re.sub(r'\b(\d)\b', lambda match: roman_numeral(int(match.group(1))), name)
        name = re.sub(r'^MV\.', '', name)
        name = re.sub(r'[^\w\s]+', '', name)  # Remove non-alphanumeric characters

        
    elif iteration == 2:
        name = re.sub(r'(\D)(\d)$', r'\1 \2', name) 
    
    elif iteration == 5:
        if len(re.findall(r'\d', name)) == 1:
            name = re.sub(r'(\D)(\d)$', r'\1 0\2', name)
            name = re.sub(r'\s+', ' ', name)  # remove multiple spaces

    elif iteration == 4:
        name = re.sub(r'\d', '', name)

    elif iteration == 3:
        match = re.search(r'(\d+)$', name)
        if match:
            name = name[:match.start()].strip() + ' ' + match.group(1)

    vessel_name = name.strip()
#     print(vessel_name)

test = 'first kasih'
process_ship_name(test, 1)
process_ship_name(test, 2)
process_ship_name(test, 3)
process_ship_name(test, 4)
process_ship_name(test, 5)



In [57]:
#data mining di website
def shipdata():
    global LOA , LPP , B , H , T
    global DWT , NT , GT , LT
    global NAME , REGISTER , IMO , status, type
    global link
    
    NAME = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[1]/table/tbody[2]/tr/td[1]/b')
    REGISTER = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[1]/table/tbody[2]/tr/td[2]/b')
    IMO = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[1]/table/tbody[2]/tr/td[3]/b')
    status = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[1]/table/tbody[2]/tr/td[4]/b')
    
    #principal dimension
    LOA = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[1]/table/tbody/tr/td[1]/b')
    LPP = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[1]/table/tbody/tr/td[2]/b')
    B = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[1]/table/tbody/tr/td[3]/b')
    H = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[1]/table/tbody/tr/td[4]/b')
    T = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[4]/td[3]')
    
    #ship tonnage
    LT = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[2]/table/tbody/tr/td[1]/b')
    GT = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[2]/table/tbody/tr/td[2]/b')
    NT = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[2]/table/tbody/tr/td[3]/b')
    DWT = driver.find_element(By.XPATH,'/html/body/div/div/div/div/div/div/div[4]/table/tbody/tr[3]/td[2]/table/tbody/tr/td[4]/b')
    
    link = 'https://www.bki.co.id/shipregister-{}.html'.format(REGISTER.text)
    # print variable names and value
    


In [58]:
# Define the fieldnames for the CSV file
fieldnames = ['NAME', 'REGISTER', 'IMO', 'status', 'LOA', 'LPP', 'B', 'H', 'T', 'LT', 'GT', 'NT', 'DWT', 'Link']

In [63]:
# Set up the webdriver
driver = webdriver.Chrome()

driver.get("https://www.bki.co.id/shipregister.html")

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()
    
    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)        
        for row in reader:
            vessel_name = row[0]
            original_name = vessel_name
            driver.get("https://www.bki.co.id/shipregister.html")
            
            try:
                search(vessel_name)
                click()
                hull()
                shipdata()
                
            except:
                try:
                    process_ship_name(vessel_name, 1)
                    search(vessel_name)
                    click()
                    hull()
                    shipdata()
                    

                except:
                    try:
                        process_ship_name(vessel_name, 2)
                        search(vessel_name)
                        click()
                        hull()
                        shipdata()
                        

                    except:
                        try:
                            process_ship_name(vessel_name, 3)
                            search(vessel_name)
                            click()
                            hull()
                            shipdata()
                            

                        except:
                            try:
                                process_ship_name(vessel_name, 4)
                                search(vessel_name)
                                click()
                                hull()
                                shipdata()
                            except:
                                
                                try:
                                    process_ship_name(vessel_name, 5)
                                    search(vessel_name)
                                    click()
                                    hull()
                                    shipdata()
                                except:
                                    print('ga ketemu kapal :', original_name)
                                    continue
                                
                try:                
                    details_dict = {'NAME': NAME.text, 'REGISTER': REGISTER.text, 'IMO': IMO.text, 'status': status.text, 'LOA': LOA.text, 'LPP': LPP.text, 'B': B.text, 'H': H.text, 'T': T.text, 'LT': LT.text, 'GT': GT.text, 'NT': NT.text, 'DWT': DWT.text, 'Link': link}
                    writer.writerow(details_dict)
                    print('ketemu kapal :', original_name)
                except:
                    continue


ga ketemu kapal : name
ga ketemu kapal : HABCO PIONEER
ketemu kapal : PUSRI INDONESIA 1
ga ketemu kapal : LUMOSO PROSPER
ga ketemu kapal : MV NAZIHA
ga ketemu kapal : DHARMA LAUTAN RUBY
ketemu kapal : CAHAYA MUSTIKA LAUT1
ga ketemu kapal : AMANAH SULAWESI AMC
ga ketemu kapal : MBS BALURAN
ga ketemu kapal : SINAR KINTAMANI
ga ketemu kapal : MANALAGI TISYA
ga ketemu kapal : ARFIANIE AYU
ga ketemu kapal : GOLDEN REJEKI
ga ketemu kapal : ASIAN WISDOM
ga ketemu kapal : LUMOSO LESTARI
ga ketemu kapal : LUMOSO KARUNIA II
ga ketemu kapal : GUNALEILA
ga ketemu kapal : DEWI AMBARWATI
ga ketemu kapal : MUMTAZ
ga ketemu kapal : LUMOSO LANCAR
ga ketemu kapal : GOLDEN ROSE
ga ketemu kapal : MUNQIDZ
ga ketemu kapal : MANALAGI WANDA
ga ketemu kapal : MV PUTERI SEJATI
ga ketemu kapal : NOAH ASYERA
ga ketemu kapal : MV. LUMOSO RAYA
ga ketemu kapal : MV.MUHASYIR
ga ketemu kapal : UNITAMA LILY
ga ketemu kapal : LUMOSO SURYA
ketemu kapal : -SARTIKA BARUNA
ga ketemu kapal : LGH PROSPER
ga ketemu kapal : MAN

In [ ]:
#GPT VERSION
# Set up the webdriver
driver = webdriver.Chrome()

driver.get("https://www.bki.co.id/shipregister.html")
vessel_name = 'KT 05'
search(vessel_name)


for i in range(1, 5):
    try:
        search(vessel_name)
        click()
        break
    except:
        print(f"mencoba pergantian iterasi {i}")
        process_ship_name(vessel_name, i)

In [ ]:
# Set up the webdriver
driver = webdriver.Chrome()

driver.get("https://www.bki.co.id/shipregister.html")

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()
    
    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        driver.get("https://www.bki.co.id/shipregister.html")
        
        for row in reader:
            vessel_name = row[0]
            try:
                search(vessel_name)
                click()
                hull()
                shipdata()
            except:
                try:
                    print('mencoba pergantian iterasi 1')
                    process_ship_name(vessel_name, 1)
                    search(vessel_name)
                    click()
                    hull()
                    shipdata()

                except:
                    try:
                        print('mencoba pergantian iterasi 2')
                        process_ship_name(vessel_name, 2)
                        search(vessel_name)
                        click()
                        hull()
                        shipdata()

                    except:
                        try:
                            print('mencoba pergantian iterasi 3')
                            process_ship_name(vessel_name, 3)
                            search(vessel_name)
                            click()
                            hull()
                            shipdata()

                        except:
                            try:
                                print('mencoba pergantian iterasi 4')
                                process_ship_name(vessel_name, 4)
                                search(vessel_name)
                                click()
                                hull()
                                shipdata()
                            except:
                                print('emang kapalnya gaada di bki')
                details_dict = {'NAME': NAME.text, 'REGISTER': REGISTER.text, 'IMO': IMO.text, 'status': status.text, 'LOA': LOA.text, 'LPP': LPP.text, 'B': B.text, 'H': H.text, 'T': T.text, 'LT': LT.text, 'GT': GT.text, 'NT': NT.text, 'DWT': DWT.text, 'Link': link}
                writer.writerow(details_dict)
